In [60]:
import pandas as pd
import json
import os
from tqdm import tqdm
import zipfile
import unidecode
from core.utils import trat_str, trat_cpf, trat_tit_eleitoral

# Encontrar o título eleitoral por parlamentar para cruzar com a base classificada

## Bases TSE

In [ ]:
folder = './data/tse/candidatos/'
files = os.listdir(folder)
df_tse = pd.DataFrame()

for file in tqdm(files):
    with zipfile.ZipFile(folder + file) as f:
        f_bra = [n for n in f.infolist()  if 'BRASIL' in n.filename][0]
        df_cand = pd.read_csv(f.open(f_bra.filename), encoding='latin1', sep=';')
        df_tse = pd.concat([df_tse, df_cand])

df_tse = df_tse[df_tse['CD_CARGO'] == 6] # somente candidatos a dep. fed.

## Econtrar id_parl para cada cpf

In [ ]:
parls = []
files = os.listdir('./data/parlamentares_detalhes/')
for file in tqdm(files):
    with open(f'./data/parlamentares_detalhes/{file}', 'r') as f:
        parls += json.load(f)
df_parls = pd.DataFrame(parls)

df_parls['cpf_trat'] = df_parls['cpf'].apply(trat_cpf)
df_tse['cpf_trat'] = df_tse['NR_CPF_CANDIDATO'].apply(trat_cpf)

df_depara = df_parls.merge(df_tse, on='cpf_trat', how='left')

df_depara = df_depara.loc[:, ['id', 'cpf', 'NR_TITULO_ELEITORAL_CANDIDATO']].drop_duplicates()
df_depara.rename({'NR_TITULO_ELEITORAL_CANDIDATO': 'n_tit_eleitoral'}, axis=1, inplace=True)

In [200]:
df_depara.to_excel('./data/depara_id_cpf_tit_eleitoral.xlsx', index=False)

## Base classificada

In [299]:
df_clf = pd.read_excel('./data/BD_dep fed 98-14 artigo ALH.xlsx')

df_clf['n_tit_eleitoral_trat'] = df_clf['n_tit_eleitoral'].apply(trat_tit_eleitoral)
df_depara['n_tit_eleitoral_trat'] = df_depara['n_tit_eleitoral'].apply(trat_tit_eleitoral)

df_final = df_clf.merge(df_depara, on='n_tit_eleitoral_trat', how='left')

df_final = df_final.rename({'id_x': 'id', 'id_y': 'id_parl', 'n_tit_eleitoral_x': 'n_tit_eleitoral' }, axis=1)

df_final = df_final[~df_final['id_parl'].isna()].reset_index(drop=True)

df_final['id_parl'] = [str(int(id_)) for id_ in df_final['id_parl']]


In [314]:
cols = ['id',
'ano_eleicao',
'n_tit_eleitoral_trat',
'id_parl',
'cpf',
'nome_candidato',
'nome_urna_cand',
'ocupacao_declarada',
'sexo',
'mulher_eleita',
'grau_instrucao',
'educacao',
'est_civil',
'situacao_conjugal',
'nacionalidade',
'sig_uf_nasc',
'mun_nasc',
'eleito_sim_ou_nao',
'tipo_eleito',
'cor',
'indio_sim_nao',
'branco_sim_nao',
'sigla_uf',
'estado',
'regiao',
'id_data_eleicao',
'cargo_disputado',
'natureza_cargo',
'situacao_candidatura',
'num_partido',
'sigla_partido',
'nome_partido',
'partidos_coligacao',
'nome_coligacao',
'ano_nascimento',
]

df_final[cols].to_excel('./data/fat_parlamentares.xlsx', index=False)

# Preparar base com votos

In [297]:
from core.tipo_voto import TipoVoto

In [225]:
votos_folder = './data/votos/'
orient_folder = './data/orientacoes/'
votos_files = os.listdir(votos_folder)

In [226]:
votos = []
orientacoes = []
for file in tqdm(votos_files):
    id_votacao = file.split('.')[0]
    try:
        with open(orient_folder + id_votacao + '.json') as f:
            orients = json.load(f)
            for orient in orients:
                orient['id_votacao'] = id_votacao
            orientacoes += orients
    except FileNotFoundError:
        continue
    with open(votos_folder + file, 'r') as f:
        vots = json.load(f)
        for vot in vots:
            vot['id_votacao'] = id_votacao
        votos += vots

100%|██████████| 130036/130036 [22:31<00:00, 96.22it/s]  


In [227]:
df_votos = pd.DataFrame(votos)
df_orient = pd.DataFrame(orientacoes)

In [236]:
# REMOVER VOTACOES CONSENSUAIS
NV_CONSENSUAL = .95
df_votos['voto_trat'] = df_votos['tipoVoto'].apply(lambda x: TipoVoto.trat_voto(x))
df_votos_dummies = df_votos.join(pd.get_dummies(df_votos['voto_trat']))
df_votos_aggr = df_votos_dummies.groupby('id_votacao').sum(numeric_only=True)
df_votos_aggr['tot_votos'] = df_votos_aggr[-1] + df_votos_aggr[0] + df_votos_aggr[1]
df_votos_aggr['%_sim'] = df_votos_aggr[1] / df_votos_aggr['tot_votos']
df_votos_aggr['is_cons'] = df_votos_aggr['%_sim'].apply(lambda x: x >= NV_CONSENSUAL or x < (1 - NV_CONSENSUAL))

votacoes_nao_consensuais = df_votos_aggr[~df_votos_aggr['is_cons']].index

In [264]:
# VERIFICAR ORIENTACOES
govs = [sigla for sigla in df_orient['siglaPartidoBloco'].unique() if ('Gov' in sigla or 'GOV' in sigla)]

In [272]:
# realizar merge apenas com orientações do governi, das votações não consensuais e que não seja "Liberado"
df_orientacoes_gov = df_orient[(df_orient['siglaPartidoBloco'].isin(govs)) & 
    (df_orient['id_votacao'].isin(votacoes_nao_consensuais)) &
    (df_orient['orientacaoVoto'] != 'Liberado')].reset_index(drop=True)
df_merge = df_votos.merge(df_orientacoes_gov, on='id_votacao', how='inner')

# VERIFICAR ORIENTAÇÃO DE ABSTENÇÃO

In [275]:
df_merge['ori_trat'] = df_merge['orientacaoVoto'].apply(lambda x : TipoVoto.trat_voto(x))

In [277]:
df_merge['gov'] = df_merge['voto_trat'] * df_merge['ori_trat']

In [279]:
df_merge['id_parl'] = [dep['id'] for dep in df_merge['deputado_']]
df_merge['ano'] = [data.split('-')[0] for data in df_merge['dataRegistroVoto']]

In [290]:
df_merge = df_merge.join(pd.get_dummies(df_merge['ori_trat']).rename({-1: 'v_opo', 0: 'v_neutro', 1:'v_gov'}, axis=1))

In [293]:
df_governismo = df_merge.groupby(['id_parl', 'ano']).agg({'id_votacao': 'count', 'v_opo': 'sum', 'v_neutro': 'sum', 'v_gov': 'sum'})
df_governismo['governismo'] = df_governismo['v_gov'] / df_governismo['id_votacao']

df_governismo.rename({'id_votacao': 'tot_votacoes'}, axis=1, inplace=True)

df_governismo = df_governismo.reset_index()

In [315]:
df_governismo.to_excel('./data/fat_governismo.xlsx', index=False)